In [ ]:
"""Import modules and define fixed parameters."""

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
from dome_connector import ShipConnector
from cmcl_jobsender import JobSender
from copy import deepcopy
import re

local = True
if local:
    # local deployment
    DOME_URL = 'https://cmcl.dome40.io/'  # DOME platform URL
    API_KEY = "6f7d3ae990.fb00950be9a1439d80c6baf515e1f112"  # Identify user
    CMCL_URL = 'http://192.168.1.171:4242/'  # URL of CMCL server
    # UUID of DOME connector for ship data
    SHIP_CONN_UUID = "fb8490c8-a71a-42c4-bd67-6052cf347f2e"
else:
    # connect to platform
    DOME_URL = 'https://nextgen.dome40.io/'
    API_KEY = "3b35fa9591.bd62b7cddfe04b26916dd6eb95214094"
    CMCL_URL = 'https://theworldavatar.io/demos/ship-emission/'
    SHIP_CONN_UUID = "0a367029-0bf1-4c75-b5e2-182cd88bdf53"

In [ ]:
"""Handle pasring query parameter."""
import os
from urllib.parse import parse_qs

query_string = os.environ.get('QUERY_STRING', '')

params = parse_qs(query_string)
    
if 'mmsi' in params:    
    QUERY_MMSI = params['mmsi'][0]
else:
    raise Exception("MMSI parameter must be supplied in the form of ?mmsi=123456789 in the HTTP request.")

In [ ]:
# search DOME for ship location data - backend

# create connectors

ship_connector = ShipConnector(DOME_URL, API_KEY, SHIP_CONN_UUID)

DF_SHIP = ship_connector.get_ship(QUERY_MMSI)
SCOPE = ship_connector.get_scope(DF_SHIP)
DEFAULT_SCOPE = deepcopy(SCOPE) # safe copy

output_dome_ship = widgets.HTML(value=ship_connector.plot_ship(DF_SHIP,SCOPE),
                                layout=widgets.Layout(width = "100%", padding='10px', margin='0px 0px 20px 0px'))

In [ ]:
# submit job to CMCL server

from uuid import uuid4

job_sender = JobSender(CMCL_URL)

# Create a text area widget
text_area_job_label = widgets.Text(
    placeholder=f"""MMSI:{QUERY_MMSI}""",
    description="""Name of simulation: """,
    layout=widgets.Layout(flex_flow='row',width='70%',padding='10px 0px 0px 0px'),
    style={'description_width': 'initial'},
    disabled=False
)

slider_job_step = widgets.IntSlider(
    value=0,
    min=1,
    max=len(DF_SHIP['date']),
    description="""Number of timesteps: """,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(flex_flow='row',width='70%'),
    style={'description_width': 'initial'},
    disabled=False
)

text_area_scope_min = widgets.Text(
    placeholder=f"({SCOPE['LAT']-SCOPE['dLAT']:.5f}, {SCOPE['LON']-SCOPE['dLON']:.5f})",
    description="""Minimum (Lat,Lon): """,
    layout=widgets.Layout(flex_flow='row',width='70%'),
    style={'description_width': 'initial'},
    disabled=False
)

text_area_scope_max = widgets.Text(
    placeholder=f"({SCOPE['LAT']+SCOPE['dLAT']:.5f}, {SCOPE['LON']+SCOPE['dLON']:.5f})",
    description="""Maximum (Lat,Lon): """,
    layout=widgets.Layout(flex_flow='row',width='70%'),
    style={'description_width': 'initial'},
    disabled=False
)

text_area_job_password = widgets.Password(
    placeholder="""******""",
    description="""Password: """,
    layout=widgets.Layout(flex_flow='row',width='70%'),
    style={'description_width': 'initial'},
    disabled=False
)

# Create a button widget for overwriting the scope
button_scope = widgets.Button(
    description='Update',
    button_style='warning',
    tooltip='Update scope',
    icon='circle'
)

def click_button_scope(b):
    
    b.button_style = 'info'  # Change the button style
    
    with output_job:
        output_job.clear_output()
        try:
            match_min = re.findall(r'-?\d+\.?\d*', text_area_scope_min.value)
            min_lat = float(match_min[0])
            min_lon = float(match_min[1])
            match_max = re.findall(r'-?\d+\.?\d*', text_area_scope_max.value)
            max_lat = float(match_max[0])
            max_lon = float(match_max[1])
            SCOPE['LAT'] = (max_lat + min_lat) * .5
            SCOPE['LON'] = (max_lon + min_lon) * .5
            SCOPE['dLAT'] = abs(max_lat - min_lat) * .5
            SCOPE['dLON'] = abs(max_lon - min_lon) * .5
            print(f"Minimum (Lat,Lon): ({SCOPE['LAT']-SCOPE['dLAT']:.5f}, {SCOPE['LON']-SCOPE['dLON']:.5f})")
            print(f"Maximum (Lat,Lon): ({SCOPE['LAT']+SCOPE['dLAT']:.5f}, {SCOPE['LON']+SCOPE['dLON']:.5f})")
        except:
            print('Invalid scope specification. Please try again.')

        output_dome_ship.value=ship_connector.plot_ship(DF_SHIP,SCOPE)
    
    b.button_style = 'warning'  # Change the button style

button_scope.on_click(click_button_scope)
        
# Create a button widget for reset the scope
button_reset_scope = widgets.Button(
    description='Reset',
    button_style='warning',
    tooltip='Reset scope',
    icon='circle'
)

def click_button_reset_scope(b):
    
    b.button_style = 'info'  # Change the button style
    
    with output_job:
        output_job.clear_output()
        SCOPE=deepcopy(DEFAULT_SCOPE)
        output_dome_ship.value=ship_connector.plot_ship(DF_SHIP,SCOPE)
    
    b.button_style = 'warning'  # Change the button style


button_reset_scope.on_click(click_button_reset_scope)

# Create a button widget for job submission
button_job = widgets.Button(
    description='Submit',
    button_style='warning',
    tooltip='Submit job',
    icon='circle'
)

def click_button_job(b):
    
    ob_style = b.button_style
    ob_description = b.description
    
    b.button_style = 'info'  # Change the button style
    b.description = 'Submitted!'  # Change the button description
    b.icon = 'check'

    with output_job:
        output_job.clear_output()
        
        if len(text_area_job_label.value)>0:
            label = text_area_job_label.value
        else:
            label = f"MMSI:{QUERY_MMSI}"
            
        try:
            num_step = slider_job_step.value
        except:
            print('Cannot parse number of steps, assume to be 0.')
            num_step = 0
        
        if num_step == 0:
            print('Number of timestep = 0 i.e. all timesteps will be simulated.')
            num_step = len(DF_SHIP['date'])
            
        
        DF_SHIP['dates'] = pd.to_datetime(DF_SHIP['date'])
        list_timestep = [x.to_pydatetime() for x in DF_SHIP['dates']]
        
        if text_area_job_password.value == os.getenv('NB_PW', uuid4()):
            print('Sending ship data...')
            response = job_sender.add_ship_data(QUERY_MMSI,DF_SHIP)
            if response.status_code == 200:
                print('Ship data successfully sent.')
                print(job_sender.run_simulation_without_ship(label,SCOPE,list_timestep,num_step))
        else:
            print('No correct password is supplied.')
            print('*** MOCK SIMULATION OUTPUT ***')
            print('Sending ship data...')
            print('Ship data successfully sent.')
            print(f'Running simulation {label}...')
            count = 0
            for t in list_timestep:
                if count < num_step:
                    print(f'Simulating {label} at {t}')
                    count = count + 1
            print('Complete.')

    b.button_style = ob_style  # Change the button style
    b.description = ob_description  # Change the button description

# Set the event handler for the button click event
button_job.on_click(click_button_job)

output_job = widgets.Output()
output_job.layout.overflow = 'auto'
output_job.layout.max_height = '200px'

In [ ]:
# final-layout

colour_bg="linear-gradient(to top left, rgba(131,195,141,0.3), rgba(200,200,200,0.3))"
colour_edge="rgba(124,203,219,1)"
css_edge="1px solid"
css = f"""
<style>
.label_style{{
    border : {css_edge};
    width:auto;
    border-radius: 10px;
    font-size:24px;
    font-weight:bold;
    color:black;
    text-align:center;
    border-color:{colour_edge};
    background: {colour_bg};
    padding: 10px 10px 10px 10px;
    margin: 0px 0px 20px 0px;
}}
.box_style{{
    border : {css_edge};
    border-radius: 20px;
    height: auto;
    max-height: 350px;
    border-color:{colour_edge};
    background: {colour_bg};
}}
</style>
"""
display(HTML(css))

label_dome_ship = widgets.HTML(f"<div>Location data of ship MMSI:{QUERY_MMSI}</div>")
label_dome_ship.add_class('label_style')

output_dome_ship.add_class('box_style')

vbox_dome_ship = widgets.VBox(
    [label_dome_ship, output_dome_ship],
    layout=widgets.Layout(width="50%",align_items='center', margin='20px'))

########################

label_job = widgets.HTML("<div>Configure and send simulation request</div>")
label_job.add_class('label_style')

hbox_scope = widgets.HBox(
    [button_scope, button_reset_scope],
    layout=widgets.Layout()
)

vbox_form = widgets.VBox(
    [text_area_scope_min, text_area_scope_max, hbox_scope,
     text_area_job_label, slider_job_step,
     text_area_job_password, button_job, output_job],
    layout=widgets.Layout(width="100%",align_items='center')
)

vbox_job = widgets.VBox(
    [label_job, vbox_form],
    layout=widgets.Layout(width="50%",align_items='center', margin='20px'))
vbox_form.add_class('box_style')

table = widgets.HBox([vbox_dome_ship, vbox_job],
                     layout=widgets.Layout(align_items='flex-start'), width='100%')

label_job = widgets.HTML("<div>DOME Showcase 1 Job Submitter</div>")
label_job.add_class('label_style')

page = widgets.VBox([label_job, table],
                    label=widgets.Layout(align_items='center', width='90%', margin='0px'))

display(page)